In [1]:
# what are the top 3 countries with the fastest drivers, on average? 

# use lap_times, milliseconds column, merge with driver id in drivers

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [13]:
lap_times_raw  = pd.read_csv("data_raw/lap_times.csv")
drivers_raw = pd.read_csv("data_raw/drivers.csv")

lap_times_drivers_merge = pd.merge(lap_times_raw[['raceId', 'driverId', 'time', 'milliseconds']],
                                   drivers_raw,
                                   on = "driverId",
                                   how = "left")

lap_times_drivers_merge


,raceId,driverId,time,milliseconds,driverRef,number,code,forename,surname,dob,nationality,url
0,841,20,1:38.109,98109,vettel,5,VET,Sebastian,Vettel,1987-07-03,German,http://en.wikipedia.org/wiki/Sebastian_Vettel
1,841,20,1:33.006,93006,vettel,5,VET,Sebastian,Vettel,1987-07-03,German,http://en.wikipedia.org/wiki/Sebastian_Vettel
2,841,20,1:32.713,92713,vettel,5,VET,Sebastian,Vettel,1987-07-03,German,http://en.wikipedia.org/wiki/Sebastian_Vettel
3,841,20,1:32.803,92803,vettel,5,VET,Sebastian,Vettel,1987-07-03,German,http://en.wikipedia.org/wiki/Sebastian_Vettel
4,841,20,1:32.342,92342,vettel,5,VET,Sebastian,Vettel,1987-07-03,German,http://en.wikipedia.org/wiki/Sebastian_Vettel
...,...,...,...,...,...,...,...,...,...,...,...,...
538116,1096,822,1:32.998,92998,bottas,77,BOT,Valtteri,Bottas,1989-08-28,Finnish,http://en.wikipedia.org/wiki/Valtteri_Bottas
538117,1096,822,1:32.995,92995,bottas,77,BOT,Valtteri,Bottas,1989-08-28,Finnish,http://en.wikipedia.org/wiki/Valtteri_Bottas
538118,1096,822,1:31.236,91236,bottas,77,BOT,Valtteri,Bottas,1989-08-28,Finnish,http://en.wikipedia.org/wiki/Valtteri_Bottas
538119,1096,822,1:30.566,90566,bottas,77,BOT,Valtteri,Bottas,1989-08-28,Finnish,http://en.wikipedia.org/wiki/Valtteri_Bottas


In [5]:
milliseconds_agg = (lap_times_drivers_merge.groupby("nationality")
                    .agg(mean_milliseconds = ('milliseconds','mean'))
                    .sort_values(by = "mean_milliseconds", ascending = True))

milliseconds_agg.iloc[:3, :]


,mean_milliseconds
nationality,
Colombian,86976.960138
Irish,89165.956182
Czech,89210.228916


In [3]:
# how do lap times vary with pit stop times? 

lap_times_raw  = pd.read_csv("data_raw/lap_times.csv")
pit_stops_raw = pd.read_csv("data_raw/pit_stops.csv")


In [4]:
lap_rename = {"time":"lap_time", "milliseconds":"lap_milliseconds"}
lap_times = lap_times_raw.rename(columns = lap_rename)

pit_rename = {"time":"pit_time", "milliseconds":"pit_milliseconds"}
pit_stops = pit_stops_raw.rename(columns = pit_rename)


In [5]:
lap_pit_merge = pd.merge(lap_times[['driverId', 'lap_time', 'lap_milliseconds']],
                         pit_stops,
                         on = "driverId",
                         how = "left")

lap_pit_merge = lap_pit_merge.sort_values("driverId", ascending = True)
